### Import libraries

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh
import csv
from bs4 import BeautifulSoup

### Define useful functions

In [77]:
def extract_review_body(review_query):
    if review_query != None:
        return review_query.find("span", itemprop="reviewBody").get_text()
    else:
        return None

def extract_review_staydate(review_query):
    if review_query != None:
        return review_query.get_text().strip('\n').strip('Stayed in ')
    else:
        return None
    
def de_emojify(inputString):
    if inputString!=None:
        return inputString.encode('ascii', 'ignore').decode('ascii')
    else:
        return inputString

def extract_hotel_feature(hotel_query):
    if hotel_query != None:
        return hotel_query.get_text().strip('\n')
    else:
        return None    

def extract_hotel_category_list(hotel_query):
    if hotel_query != None:
        return hotel_query.find_all(class_='review_score_name')
    else:
        return None  

def extract_hotel_value_list(hotel_query):
    if hotel_query != None:
        return hotel_query.find_all(class_='review_score_value')
    else:
        return None
    
def build_rating_breakdown_list(cat_list,value_list):
    breakdown_list = []
    if cat_list != None:
        for idx in range(len(cat_list)):
            breakdown_list.append([cat_list[idx].get_text(),value_list[idx].get_text()])
    return breakdown_list

### Specify user agent

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}

### Get names and urls of hotels in NYS

In [5]:
next_page=True
ip=1
hotel_results_offset=0
hotel_info_list = []
while next_page:    
    url= 'http://www.booking.com/reviews/region/new-york-state.html?offset='+str(hotel_results_offset)+'&'
    response = requests.get(url,headers=headers)
    hotel_info = BeautifulSoup(response.text, 'html.parser')
    hotel_info_page = hotel_info.find_all(class_=['rlp-main-hotel','li'])
    hotel_info_list.extend(hotel_info_page)
    ip+=1
    hotel_results_offset+=30
    if hotel_info_page==[]:
        next_page=False

In [36]:
#hotel_url_list = []
#hotel_name_list = []
nys_hotel_info_list_csvname = './datasets/nys_hotel_info_list.csv'

with open(nys_hotel_info_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','hotel_url'])
    
    for hotel_info_item in hotel_info_list:   
        hotel_name = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link').get_text()
        hotel_url = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link')['href'].split('.html?')[0]

        #hotel_name_list.append(hotel_name)
        #hotel_url_list.append(hotel_url)
        writer.writerow([hotel_name,hotel_url])

### Get names and urls of hotels in Arizona

In [62]:
next_page=True
ip=1
hotel_results_offset=0
hotel_info_list = []
while next_page:    
    url= 'http://www.booking.com/reviews/region/arizona.html?offset='+str(hotel_results_offset)+'&'
    response = requests.get(url,headers=headers)
    hotel_info = BeautifulSoup(response.text, 'html.parser')
    hotel_info_page = hotel_info.find_all(class_=['rlp-main-hotel','li'])
    hotel_info_list.extend(hotel_info_page)
    ip+=1
    hotel_results_offset+=30
    if hotel_info_page==[]:
        next_page=False

In [63]:
#hotel_url_list = []
#hotel_name_list = []
arizona_hotel_info_list_csvname = './datasets/arizona_hotel_info_list.csv'

with open(arizona_hotel_info_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','hotel_url'])
    
    for hotel_info_item in hotel_info_list:   
        hotel_name = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link').get_text()
        hotel_url = hotel_info_item.find(class_ = 'rlp-main-hotel__hotel-name-link')['href'].split('.html?')[0]

        #hotel_name_list.append(hotel_name)
        #hotel_url_list.append(hotel_url)
        writer.writerow([hotel_name,hotel_url])

### Get reviews of hotels in NYS

In [4]:
nys_hotel_info_list_csvname = './datasets/nys_hotel_info_list.csv'
df_nys_hotel_info_list = pd.read_csv(nys_hotel_info_list_csvname)

In [234]:
nys_hotel_reviews_list_csvname = './datasets/sample3_nys_hotel_reviews_list.csv'

with open(nys_hotel_reviews_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','review_date','review_item_user_review_count',
                     'review_score_badge','review_item_header_content','review_info_tag','review_staydate',
                     'review_pos','review_neg'])
    
    for idx,hotel_url_item in enumerate(df_nys_hotel_info_list['hotel_url'][:1000]):

        next_page=True
        ip=1
        hotel_reviews_list = []
        while next_page:
            url= 'http://www.booking.com/reviews/us/hotel/'+hotel_url_item.split('/hotel/us/')[1]+'.html?page='+str(ip)+';r_lang=en'
            response = requests.get(url,headers=headers)
            hotel_reviews = BeautifulSoup(response.text, 'html.parser')
            hotel_reviews_page = hotel_reviews.find_all(class_=['review_item','li'])
            hotel_reviews_list.extend(hotel_reviews_page)
            ip+=1
            if hotel_reviews_page==[]:
                next_page=False
            
        for hotel_reviews_item in hotel_reviews_list:   
            review_date = hotel_reviews_item.find(class_ = 'review_item_date').get_text().strip('\n').strip('Reviewed: ')
            #reviewer_name = hotel_reviews_item.find(class_ = 'reviewer_name').find("span", itemprop="name").get_text()
            #reviewer_country = hotel_reviews_item.find(class_ = 'reviewer_country').find("span", itemprop="name").get_text()
            review_item_user_review_count = hotel_reviews_item.find(class_ = 'review_item_user_review_count').get_text().strip('\n')
            review_score_badge = hotel_reviews_item.find(class_ = 'review-score-badge').get_text().strip('\"')
            review_item_header_content = hotel_reviews_item.find(class_ = 'review_item_header_content').get_text().strip('\n').replace('\u201c','').replace('\u201d','')
            review_info_tag = [tag.get_text().strip('\n')[2:] for tag in hotel_reviews_item.find_all(class_=['review_info_tag','li'])]
            review_staydate = extract_review_staydate(hotel_reviews_item.find(class_ = 'review_staydate'))
            review_pos = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_pos')))
            review_neg = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_neg')))
        
            writer.writerow([df_nys_hotel_info_list['hotel_name'][idx],review_date,
                             review_item_user_review_count,review_score_badge,review_item_header_content,
                             review_info_tag,review_staydate,review_pos,review_neg])

### Get reviews of hotels in Arizona

In [5]:
arizona_hotel_info_list_csvname = './datasets/arizona_hotel_info_list.csv'
df_arizona_hotel_info_list = pd.read_csv(arizona_hotel_info_list_csvname)

In [67]:
arizona_hotel_reviews_list_csvname = './datasets/sample1_arizona_hotel_reviews_list.csv'

with open(arizona_hotel_reviews_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','review_date','review_item_user_review_count',
                     'review_score_badge','review_item_header_content','review_info_tag','review_staydate',
                     'review_pos','review_neg'])
    
    for idx,hotel_url_item in enumerate(df_arizona_hotel_info_list['hotel_url'][300:800]):

        next_page=True
        ip=1
        hotel_reviews_list = []
        while next_page:
            url= 'http://www.booking.com/reviews/us/hotel/'+hotel_url_item.split('/hotel/us/')[1]+'.html?page='+str(ip)+';r_lang=en'
            response = requests.get(url,headers=headers)
            hotel_reviews = BeautifulSoup(response.text, 'html.parser')
            hotel_reviews_page = hotel_reviews.find_all(class_=['review_item','li'])
            hotel_reviews_list.extend(hotel_reviews_page)
            ip+=1
            if hotel_reviews_page==[]:
                next_page=False
            
        for hotel_reviews_item in hotel_reviews_list:   
            review_date = hotel_reviews_item.find(class_ = 'review_item_date').get_text().strip('\n').strip('Reviewed: ')
            #reviewer_name = hotel_reviews_item.find(class_ = 'reviewer_name').find("span", itemprop="name").get_text()
            #reviewer_country = hotel_reviews_item.find(class_ = 'reviewer_country').find("span", itemprop="name").get_text()
            review_item_user_review_count = hotel_reviews_item.find(class_ = 'review_item_user_review_count').get_text().strip('\n')
            review_score_badge = hotel_reviews_item.find(class_ = 'review-score-badge').get_text().strip('\"')
            review_item_header_content = hotel_reviews_item.find(class_ = 'review_item_header_content').get_text().strip('\n').replace('\u201c','').replace('\u201d','')
            review_info_tag = [tag.get_text().strip('\n')[2:] for tag in hotel_reviews_item.find_all(class_=['review_info_tag','li'])]
            review_staydate = extract_review_staydate(hotel_reviews_item.find(class_ = 'review_staydate'))
            review_pos = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_pos')))
            review_neg = de_emojify(extract_review_body(hotel_reviews_item.find(class_ = 'review_neg')))
        
            writer.writerow([df_arizona_hotel_info_list['hotel_name'][idx],review_date,
                             review_item_user_review_count,review_score_badge,review_item_header_content,
                             review_info_tag,review_staydate,review_pos,review_neg])

### Get detailed info of hotels in NYS

In [78]:
nys_hotel_detailed_info_list_csvname = './datasets/nys_hotel_detailed_info_list_2.csv'

with open(nys_hotel_detailed_info_list_csvname, 'w', newline='', encoding="UTF-8") as f:
    writer = csv.writer(f)
    writer.writerow(['hotel_name','hotel_stars','hotel_address',
                     'hotel_overall_rating','hotel_rating_breakdown','hotel_room_name','hotel_room_capacity','hotel_room_price'])
    
    for idx,hotel_url_item in enumerate(df_nys_hotel_info_list['hotel_url']):
        
        url= 'http://www.booking.com'+ hotel_url_item +'.html?checkin=2020-10-12;checkout=2020-10-13'
        response = requests.get(url,headers=headers)
        hotel_info = BeautifulSoup(response.text, 'html.parser')
        
        hotel_stars = extract_hotel_feature(hotel_info.find(class_='hp__hotel_ratings').find(class_='invisible_spoken'))
        hotel_address = hotel_info.find(class_='hp_address_subtitle js-hp_address_subtitle jq_tooltip').get_text().strip('\n')
        hotel_room_name = extract_hotel_feature(hotel_info.find(class_='hprt-roomtype-icon-link'))
        hotel_room_capacity = extract_hotel_feature(hotel_info.find(class_='c-occupancy-icons hprt-occupancy-occupancy-info'))
        hotel_room_price = extract_hotel_feature(hotel_info.find(class_='bui-price-display__value prco-text-nowrap-helper prco-font16-helper'))
        
        url= 'http://www.booking.com/reviews/us/hotel/'+hotel_url_item.split('/hotel/us/')[1]+'.html'
        response = requests.get(url,headers=headers)
        hotel_info = BeautifulSoup(response.text, 'html.parser')
        
        hotel_overall_rating = extract_hotel_feature(hotel_info.find(class_='review-score-badge'))
        hotel_rating_breakdown_category_list = extract_hotel_category_list(hotel_info.find(class_='review_score_breakdown_list'))
        hotel_rating_breakdown_value_list = extract_hotel_value_list(hotel_info.find(class_='review_score_breakdown_list'))
        hotel_rating_breakdown = build_rating_breakdown_list(hotel_rating_breakdown_category_list,hotel_rating_breakdown_value_list)
        
        writer.writerow([df_nys_hotel_info_list['hotel_name'][idx],hotel_stars,hotel_address,
                         hotel_overall_rating,hotel_rating_breakdown,hotel_room_name,hotel_room_capacity,
                         hotel_room_price])

AttributeError: 'NoneType' object has no attribute 'find'